In [2]:
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from anndata import AnnData
import MENDER
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn import svm
from scipy import stats
import re
import os
import glob
import tifffile
import pandas as pd
from skimage.measure import regionprops_table
import numpy as np
import plotly.graph_objects as go
import scipy.stats as st
from scipy import stats

# Load data

In [ ]:
# Directory containing your segmented mask TIFF files
mask_dir = r'TNBC_shareCellData'
# Output CSV file path
output_csv = "cell_centroids.csv"

records = []

# Loop through each TIFF mask
for mask_path in glob.glob(os.path.join(mask_dir, "*.tiff")):
    # Load the mask (each cell labeled by a unique integer)
    mask = tifffile.imread(mask_path)

    if ('p30_labeledcellData' in mask_path):
        print('bruh')
    print("Shape:", mask.shape)          # e.g. (1024, 1024)
    print("Data type:", mask.dtype)      # e.g. uint16
    labels = np.unique(mask)
    print(f"Found {len(labels)-1} objects (ignoring background=0)")
    print("Some label values:", labels[:10], "…", labels[-10:])
    # Extract properties: label ID and centroids
    props = regionprops_table(
        mask,
        properties=('label', 'centroid')
    )
    
    # Convert to DataFrame
    df = pd.DataFrame(props)
    # Keep track of source image
    
    
    nums = re.search(r"\d+", os.path.basename(mask_path))
    df['image'] = int(nums.group()) if nums else None
    records.append(df)

# Concatenate all image DataFrames
all_cells = pd.concat(records, ignore_index=True)

# Rename columns for clarity
all_cells = all_cells.rename(
    columns={
        'label': 'CellID',
        'centroid-0': 'CentroidY',
        'centroid-1': 'CentroidX'
    }
)

# Save to CSV
all_cells.to_csv(output_csv, index=False)
print(f"Saved {len(all_cells)} cell centroids to {output_csv}")


Shape: (2048, 2048)
Data type: uint16
Found 5213 objects (ignoring background=0)
Some label values: [0 1 2 3 4 5 6 7 8 9] … [5204 5205 5206 5207 5208 5209 5210 5211 5212 5213]
Shape: (2048, 2048)
Data type: uint16
Found 6140 objects (ignoring background=0)
Some label values: [0 1 2 3 4 5 6 7 8 9] … [6131 6132 6133 6134 6135 6136 6137 6138 6139 6140]
Shape: (2048, 2048)
Data type: uint16
Found 8772 objects (ignoring background=0)
Some label values: [0 1 2 3 4 5 6 7 8 9] … [8763 8764 8765 8766 8767 8768 8769 8770 8771 8772]
Shape: (2048, 2048)
Data type: uint16
Found 9738 objects (ignoring background=0)
Some label values: [0 1 2 3 4 5 6 7 8 9] … [9729 9730 9731 9732 9733 9734 9735 9736 9737 9738]


In [ ]:
print(all_cells.columns)

Index(['CellID', 'CentroidY', 'CentroidX', 'image'], dtype='object')


Perform Random subsampling to decreace dataset size

In [ ]:
# Read in the full cell‐level table
df = pd.read_csv("TNBC_shareCellData/cellData.csv", sep=",")

# Choose how many cells to keep per patient:
# The patient with the fewest cells in the original dataset had 2217
# The mean is 5923.55
# The most cells is 9738
# n = 2217
n = 300

# 3. Option A: if you only want patients who have ≥ n cells, and you want exactly n per patient:
#    (patients with fewer than n cells are dropped entirely)
df_filtered = df.groupby("SampleID").filter(lambda sub: len(sub) >= n)
df_subsample = (
    df_filtered
    .groupby("SampleID", group_keys=False)
    .sample(n=n, random_state=42)
)


# Write out a new CSV (e.g. “cellData_100.csv”)
out_name = f"TNBC_shareCellData/cellData_{n}.csv"
df_subsample.to_csv(out_name, sep=",", index=False)
print(f"Wrote subsampled table with {n} cells per patient to:\n  {out_name}")

Wrote subsampled table with 300 cells per patient to:
  TNBC_shareCellData/cellData_300.csv


In [ ]:
df = pd.read_csv(f"TNBC_shareCellData/cellData_{n}.csv", sep=",")
df_p = pd.read_csv("TNBC_shareCellData/patient_class.csv", header=None)
df_p.rename(columns={0:"SampleID", 1:"subtype"}, inplace=True)
subtype_map = {
    0: "Mixed",
    1: "Compartimentalized",
    2: "Cold"
}

df_p["subtype"] = df_p["subtype"].map(subtype_map).astype('category')



# TO DO: LOOK INTO THIS FILTERING SINCE SAMPLEID 30 DOESN"T ACTUALLY EXIST SO MAYBE IT IS MORE COMPLICATED THAN JUST REMOVING A FEW
bad_Values = [42,43,44]

df = df[~df["SampleID"].isin(bad_Values)]


meta_cols = ['SampleID', 'cellLabelInImage', 'cellSize', 'tumorYN', 
             'tumorCluster', 'Group', 'immuneCluster', 'immuneGroup']

expr_cols = [c for c in df.columns if c not in meta_cols]

adata = AnnData(
    X = df[expr_cols].values,
    obs = df[meta_cols],
    var=pd.DataFrame(index=expr_cols)
)

adata.obs["UniqueID"] = adata.obs['SampleID'].astype(str) + "_" + adata.obs['cellLabelInImage'].astype(str)
all_cells["UniqueID"] = all_cells["image"].astype(str) + "_" + all_cells["CellID"].astype(str)
all_cells = all_cells.set_index('UniqueID', drop = False)
adata.obs = adata.obs.set_index('UniqueID', drop=False)

all_cells_aligned = all_cells.reindex(adata.obs_names)

adata.obsm["spatial"] = all_cells_aligned[["CentroidX", "CentroidY"]].to_numpy()

# X = adata.X.toarray() if hasattr(adata.X, "toarray") else adata.X
# min_val = X.min()
# print("Minimum before shift:", min_val)

# 2. If it’s < 0, shift everything up so the minimum is 0
# if min_val < 0:
#     shift = -min_val
#     print(f"Shifting all values by +{shift:.3g} to eliminate negatives")
#     X += shift
#     adata.X = X  # put it back into your AnnData

# sc.pp.normalize_total(adata)         # e.g. counts per cell
# sc.pp.log1p(adata)
# sc.pp.scale(adata)

# sc.tl.pca(adata, svd_solver='arpack', n_comps=50)
# sc.pp.neighbors(adata)               # builds graph on PCA
# sc.tl.umap(adata)
code2label_group = {
    1: "Unidentified",
    2: "Immune",
    3: "EndoThelial",
    4: "Mesenchymal-like",
    5: "Tumor",
    6: "Keratin-positive tumor"
}

code2label_immunegroup = {
    1: "Tregs",
    2: "CD4 T",
    3: "CD8 T",
    4: "CD3 T",
    5: "NK",
    6: "B",
    7: "Neutrophils",
    8: "Macrophages",
    9: "DC",
    10: "DC/Mono",
    11: "Mono/Neu",
    12: "Other immune"
}
adata.obs["Group"] = adata.obs["Group"].map(code2label_group).astype('category')
adata.obs["immuneGroup"] = adata.obs["immuneGroup"].map(code2label_immunegroup)
adata.obs = pd.merge(
    adata.obs,
    df_p,
    on="SampleID",
    how="left"
)

# Copy for later use
adata_raw = adata.copy()

adata.obs

c:\Users\chris\anaconda3\envs\ml_for_bio_3.9\lib\site-packages\anndata\_core\anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
c:\Users\chris\anaconda3\envs\ml_for_bio_3.9\lib\site-packages\anndata\_core\anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,SampleID,cellLabelInImage,cellSize,tumorYN,tumorCluster,Group,immuneCluster,immuneGroup,UniqueID,subtype
0,1,1839,230,0,0,Immune,85,CD4 T,1_1839,Mixed
1,1,3381,361,0,0,Immune,84,B,1_3381,Mixed
2,1,3934,121,0,0,Immune,85,B,1_3934,Mixed
3,1,3648,187,0,0,Immune,75,B,1_3648,Mixed
4,1,1639,567,1,0,Keratin-positive tumor,0,NaN,1_1639,Mixed
...,...,...,...,...,...,...,...,...,...,...
11995,41,50,625,0,0,Immune,57,CD8 T,41_50,Compartimentalized
11996,41,3723,380,1,16,Mesenchymal-like,0,NaN,41_3723,Compartimentalized
11997,41,227,340,0,0,Immune,45,Mono/Neu,41_227,Compartimentalized
11998,41,860,801,0,0,Immune,47,CD3 T,41_860,Compartimentalized


# Coarse representation

In [ ]:
# Compute “coarse” counts per SampleID × Group, but KEEP SampleID as the index
coarse = (
    adata.obs
         .groupby(["SampleID", "Group"])
         .size()
         .unstack(fill_value=0)
)
# Now coarse.index is SampleID

# Normalize each row to percentages
coarse_norm = coarse.div(coarse.sum(axis=1), axis=0)
# coarse_norm.index is still SampleID

# Build y so that it lines up with coarse_norm.index 
y = adata.obs.groupby("SampleID")["subtype"].first().loc[coarse_norm.index]

# Run KNN with cross‐validation
knn = KNeighborsClassifier()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
coarse_knn = cross_val_score(knn, coarse_norm, y, scoring="accuracy", cv=cv)

print(coarse_knn)
print(coarse_knn.mean(), coarse_knn.std())

# UNCOMMENT TO SHOW COARSE REPRESENTATION
# coarse_norm

[0.75  0.5   0.875 0.75  0.625 0.875 0.75  0.75  0.875 0.75  0.625 0.75
 0.75  0.875 0.75  1.    0.75  0.75  0.875 0.625 0.625 0.75  0.625 0.75
 0.625 0.625 0.75  0.875 0.625 0.75  0.75  0.875 0.75  0.5   0.5   0.75
 0.625 0.625 0.875 0.625 0.75  0.75  0.625 0.625 0.75  0.875 0.75  0.625
 0.75  0.75 ]
0.7275 0.10807983160608645


In [ ]:
svc = svm.SVC()
coarse_svc = cross_val_score(svc, coarse_norm, y, scoring='accuracy', cv=cv)
print(coarse_svc)
coarse_svc.mean(), coarse_svc.std()

[0.75  0.625 0.875 0.875 0.75  0.75  0.875 0.75  0.75  0.625 0.625 0.75
 0.75  0.875 0.625 0.875 0.75  0.75  0.75  0.75  0.875 0.75  0.75  0.75
 0.625 0.75  0.875 0.75  0.75  0.75  0.75  0.875 0.875 0.75  0.5   0.75
 0.875 0.5   0.75  0.625 0.875 0.875 0.75  0.625 0.75  0.75  0.75  0.625
 0.875 0.75 ]


(0.7525, 0.0951643315533714)

# Fine representation

In [ ]:
data_fine = adata.obs.copy()
data_fine.loc[data_fine["immuneGroup"].isna(), "immuneGroup"] = data_fine.loc[data_fine["immuneGroup"].isna(), "Group"]

In [ ]:
fine = data_fine.groupby(["SampleID", "immuneGroup"]).size().unstack(fill_value=0).reset_index(drop=True)
# Normalize the counts to percentages
fine_norm = fine.div(fine.sum(axis=1), axis=0)
# UNCOMMENT TO SHOW FINE REPRESENTAITON
# fine

In [ ]:
knn = KNeighborsClassifier()
fine_knn = cross_val_score(knn, fine_norm, y, scoring='accuracy', cv=cv)
print(fine_knn)
fine_knn.mean(), fine_knn.std()

[0.625 0.625 0.75  0.875 0.75  0.75  0.875 0.875 0.75  0.625 0.75  0.75
 0.75  0.875 0.625 0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.875
 0.625 0.625 0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.625 0.625 0.875
 0.875 0.75  0.75  0.625 0.875 0.75  0.75  0.625 0.75  0.75  0.875 0.625
 0.75  0.75 ]


(0.745, 0.0788986691902975)

In [ ]:
svc = svm.SVC()
fine_svc = cross_val_score(svc, fine_norm, y, scoring='accuracy', cv=cv)
print(fine_svc)
fine_svc.mean(), fine_svc.std()

[0.625 0.625 0.875 0.875 0.75  0.75  0.875 0.875 0.75  0.625 0.75  0.75
 0.75  0.875 0.75  0.875 0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.875
 0.625 0.75  0.875 0.75  0.75  0.75  0.75  0.875 0.75  0.75  0.625 0.875
 0.875 0.75  0.75  0.625 0.875 0.875 0.75  0.625 0.75  0.75  0.875 0.625
 0.875 0.75 ]


(0.7675, 0.08295330011518529)

# MENDER representation

In [ ]:
# batch_obs = 'subtype'
batch_obs = 'SampleID'
scale = 6
radius = 15

# I'm not sure why they did so many copies but I took it from this code 
# https://mender-tutorial.readthedocs.io/en/latest/MERSCOPE.html
adata = adata_raw.copy()

# adata.obs['SampleID'] = adata.obs['SampleID'].astype('category')

# Only do it for these 18 samples to make it run more quickly. 6 mixed, comp and cold
# sample_ids = [5, 13, 1, 2, 3, 4]
# sample_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 15, 19, 22, 24, 25, 26]
sample_ids = [
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,  22, 23, 24, 25, 26,
    27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41,
]

adata = adata[adata.obs['SampleID'].isin(sample_ids)].copy()

adata.obs['SampleID'] = adata.obs['SampleID'].astype('category')


# TODO: I think the grouping here should be based on fine groups
# main body of MENDER
msm = MENDER.MENDER(
    adata,
    batch_obs = batch_obs,
    # determine which cell state to use
    # In our case the cell type does not need to be estimated since we have a ground truth stored in 
    # the dataset under all_group_name
    ct_obs= 'Group',
    random_seed=42,
    verbose=True
)


# set the MENDER parameters


msm.prepare()
msm.set_MENDER_para(
    # default of n_scales is 6
    n_scales=scale,

    # for single cell data, nn_mode is set to 'radius'
    nn_mode='radius',

    # default of n_scales is 15 um (see the manuscript for why).
    # MENDER also provide a function 'estimate_radius' for estimating the radius
    nn_para=radius,
)


# construct the context representation
msm.run_representation_mp(
    50
    # the number of processings
)

# set the spatial clustering parameter
# positive values for the expected number of domains
# negative values for the clustering resolution
cluster_parameter = 13
# msm.run_clustering_normal(-0.5)
# msm.run_clustering_normal(2)
msm.run_clustering_normal(cluster_parameter)
msm.adata_MENDER.obs


NameError: name 'adata_raw' is not defined

In [ ]:

# Compute the raw counts per SampleID × MENDER, then normalize per sample
mender = (
    msm.adata_MENDER.obs
       .groupby(['SampleID', 'MENDER'])
       .size()
       .unstack(fill_value=0)
)
mender_norm = mender.div(mender.sum(axis=1), axis=0).reset_index()  # SampleID becomes a column

# Extract y_raw (subtype) for each SampleID, in the same sample_ids order
y_raw = adata_raw.obs.groupby('SampleID')['subtype'].first()
y_raw = y_raw.loc[sample_ids]  # reorder so it matches sample_ids

# Turn y_raw into a DataFrame so we can merge on SampleID
y_df = y_raw.reset_index().rename(columns={'subtype': 'subtype'})  # columns: ['SampleID','subtype']

# Merge the normalized MENDER fractions with the subtype column
merged = mender_norm.merge(y_df, on='SampleID', how='left')

file_path = f"mender_representation_{n}_samples_{cluster_parameter}_cluster_parameter.csv"
# Write the final table to CSV (includes SampleID, MENDER‐normalized columns, and subtype)
merged.to_csv(file_path, index=False)


Read saved mender represation and run classification

In [ ]:
# Read the merged CSV back into a DataFrame
df = pd.read_csv(file_path)

# Put SampleID back into the index (optional, but often convenient)
df = df.set_index("SampleID")

# Extract y_raw as a Series of subtype values
y_raw = df["subtype"]

# Extract mender_norm by dropping the “subtype” column
mender_norm = df.drop(columns=["subtype"])

In [ ]:
# Define your classifier
knn = KNeighborsClassifier()

# Compute cross‐val scores over all 10×5=50 folds at once
mender_knn = cross_val_score(knn, mender_norm, y_raw, scoring="accuracy", cv=cv)

print(mender_knn)
mender_knn.mean(), mender_knn.std()

[0.5   0.875 0.875 0.75  0.375 0.75  0.5   1.    0.625 0.75  0.625 0.75
 0.75  0.625 0.875 0.75  0.625 0.75  0.625 0.75  0.75  0.875 0.625 0.75
 0.625 1.    0.625 0.75  0.625 0.5   0.625 0.875 0.625 0.75  0.625 0.75
 0.625 0.625 0.75  0.75  0.625 0.75  0.625 0.75  0.625 0.625 0.875 0.5
 0.75  0.5  ]


(0.6975, 0.12769592789122133)

In [ ]:
svc = svm.SVC()
# Define your classifier

# Use RepeatedStratifiedKFold to get 10 different random 5‐fold splits
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

# Compute cross‐val scores over all 10×5=50 folds at once
mender_svc = cross_val_score(svc, mender_norm, y_raw, scoring="accuracy", cv=cv)

print(mender_svc)
mender_svc.mean(), mender_svc.std()

[0.5   0.5   0.5   0.5   0.375 0.5   0.5   0.5   0.5   0.375 0.5   0.5
 0.5   0.5   0.375 0.5   0.5   0.5   0.5   0.375 0.5   0.5   0.5   0.5
 0.375 0.5   0.5   0.5   0.5   0.375 0.5   0.5   0.5   0.5   0.375 0.5
 0.5   0.5   0.5   0.375 0.5   0.5   0.5   0.5   0.375 0.5   0.5   0.5
 0.5   0.375]


(0.475, 0.049999999999999996)

# T-test

## KNN

In [ ]:
# By default, ttest_ind returns a two-sided p-value
t_stat, p_two_sided = stats.ttest_ind(mender_knn, fine_knn, equal_var=False)

# Convert to one-sided p-value
if t_stat > 0:
    p_one_sided = p_two_sided / 2
else:
    p_one_sided = 1 - p_two_sided / 2

print(f"t-statistic: {t_stat:.4f}")
print(f"one-sided p-value: {p_one_sided:.4f}")
p_knn_mender_fine = p_one_sided

t-statistic: -1.0315
one-sided p-value: 0.8228


In [ ]:
# By default, ttest_ind returns a two-sided p-value
t_stat, p_two_sided = stats.ttest_ind(mender_knn, coarse_knn, equal_var=False)

# Convert to one-sided p-value
if t_stat > 0:
    p_one_sided = p_two_sided / 2
else:
    p_one_sided = 1 - p_two_sided / 2

print(f"t-statistic: {t_stat:.4f}")
print(f"one-sided p-value: {p_one_sided:.4f}")
p_knn_mender_coarse = p_one_sided

t-statistic: -0.8276
one-sided p-value: 0.7744


## SVM

In [ ]:
# By default, ttest_ind returns a two-sided p-value
t_stat, p_two_sided = stats.ttest_ind(mender_svc, fine_svc, equal_var=False)

# Convert to one-sided p-value
if t_stat > 0:
    p_one_sided = p_two_sided / 2
else:
    p_one_sided = 1 - p_two_sided / 2

print(f"t-statistic: {t_stat:.4f}")
print(f"one-sided p-value: {p_one_sided:.4f}")
p_svc_mender_fine = p_one_sided

t-statistic: -4.8660
one-sided p-value: 0.9962


In [ ]:
# By default, ttest_ind returns a two-sided p-value
t_stat, p_two_sided = stats.ttest_ind(mender_svc, coarse_svc, equal_var=False)

# Convert to one-sided p-value
if t_stat > 0:
    p_one_sided = p_two_sided / 2
else:
    p_one_sided = 1 - p_two_sided / 2

print(f"t-statistic: {t_stat:.4f}")
print(f"one-sided p-value: {p_one_sided:.4f}")
p_svc_mender_coarse = p_one_sided

t-statistic: -4.8660
one-sided p-value: 0.9962


# Visualize results

In [ ]:
# Calculate 95% confidence intervals for each group
def mean_ci(data, confidence=0.95):
    n = len(data)
    m = np.mean(data)
    se = st.sem(data)
    h = se * st.t.ppf((1 + confidence) / 2., n-1)
    return m, h

means = []
cis = []
for arr in [mender_knn, fine_knn, coarse_knn]:
    m, h = mean_ci(arr)
    means.append(m)
    cis.append(h)

bar_names = ['MENDER', 'Fine', 'Coarse']

fig = go.Figure(
    data=[
        go.Bar(
            x=bar_names,
            y=means,
            error_y=dict(type='data', array=cis, visible=True, color='black', thickness=2, width=8),
            marker_color=['#636EFA', '#EF553B', '#00CC96'],
            showlegend=False
        )
    ]
)

# Add individual data points
all_knn = [mender_knn, fine_knn, coarse_knn]
for i, arr in enumerate(all_knn):
    fig.add_trace(
        go.Scatter(
            x=[bar_names[i]] * len(arr),
            y=arr,
            mode='markers',
            marker=dict(color='black', size=8),
            name='Data points',
            showlegend=False
        )
    )

# Add p-value annotations above the Fine and Coarse bars
fig.add_annotation(
    x='Fine',
    y=means[1] + 0.05,
    text=f"p = {p_knn_mender_fine:.2f}",
    showarrow=False,
    font=dict(size=14),
    xanchor='left'
)
fig.add_annotation(
    x='Coarse',
    y=means[2] + 0.05,
    text=f"p = {p_knn_mender_coarse:.2f}",
    showarrow=False,
    font=dict(size=14),
    xanchor='left'
)

fig.update_layout(
    title={'text': 'KNN Accuracy Comparison (with 95% CI)', 'x': 0.5, 'xanchor': 'center'},
    yaxis_title='Accuracy (mean ± 95% CI)',
    xaxis_title='Representation',
    yaxis=dict(range=[0, 1]),
    template='plotly_white',
    width=500,
    height=450
)
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed